# Connecting to the ChEMBL MySQL Database with Python
## ------For Giardia intestinalis (Ceramide glucosyltransferase)------

Use MySQL Connector and Pandas to retrieve and manipulate the data.<br/>
To install mysql-connector run: `!pip install mysql-connector-python-rf`. <br/>

In [1]:
import mysql.connector as sql
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
# USER='tcrd'
# PASSWORD='tcrd'
# HOST='129.108.3.163'
# DB='chembl25'  # for chembl and 'tcrd541' for pharos

## Connect to the database (UTEP Server using VPN)

In [3]:
db_connection = sql.connect(host='129.108.3.163', db='chembl25', user='tcrd', password='tcrd')
db_connection

In order to use the new connnection, we need to create a cursor object. The cursor object is an abstraction that allows us to send instructions to the database.

In [4]:
db_cursor = db_connection.cursor()

## Executing database queries

First we execute the `SHOW TABLES;` MySQL command, to see which kind of tables we can collect information from. The `cursor.fetchall()` method returns a list.

In [5]:
db_cursor.execute('SHOW TABLES;')
tables = db_cursor.fetchall()
print(tables)

[('action_type',), ('activities',), ('activity_properties',), ('activity_smid',), ('activity_stds_lookup',), ('activity_supp',), ('activity_supp_map',), ('assay_class_map',), ('assay_classification',), ('assay_parameters',), ('assay_type',), ('assays',), ('atc_classification',), ('binding_sites',), ('bio_component_sequences',), ('bioassay_ontology',), ('biotherapeutic_components',), ('biotherapeutics',), ('cell_dictionary',), ('chembl_id_lookup',), ('component_class',), ('component_domains',), ('component_go',), ('component_sequences',), ('component_synonyms',), ('compound_properties',), ('compound_records',), ('compound_structural_alerts',), ('compound_structures',), ('confidence_score_lookup',), ('curation_lookup',), ('data_validity_lookup',), ('defined_daily_dose',), ('docs',), ('domains',), ('drug_indication',), ('drug_mechanism',), ('formulations',), ('frac_classification',), ('go_classification',), ('hrac_classification',), ('indication_refs',), ('irac_classification',), ('ligand

In [6]:
db_cursor.execute('DESCRIBE compound_structures;')
list(db_cursor)

[('molregno', 'bigint(20)', 'NO', 'PRI', None, ''),
 ('molfile', 'mediumtext', 'YES', '', None, ''),
 ('standard_inchi', 'varchar(4000)', 'YES', '', None, ''),
 ('standard_inchi_key', 'varchar(27)', 'NO', 'UNI', None, ''),
 ('canonical_smiles', 'varchar(4000)', 'YES', '', None, '')]

In [7]:
db_cursor.execute('DESCRIBE activities')
list(db_cursor)

[('activity_id', 'bigint(20)', 'NO', 'PRI', None, ''),
 ('assay_id', 'bigint(20)', 'NO', 'MUL', None, ''),
 ('doc_id', 'bigint(20)', 'YES', 'MUL', None, ''),
 ('record_id', 'bigint(20)', 'NO', 'MUL', None, ''),
 ('molregno', 'bigint(20)', 'YES', 'MUL', None, ''),
 ('standard_relation', 'varchar(50)', 'YES', 'MUL', None, ''),
 ('standard_value', 'decimal(64,30)', 'YES', 'MUL', None, ''),
 ('standard_units', 'varchar(100)', 'YES', 'MUL', None, ''),
 ('standard_flag', 'tinyint(4)', 'YES', '', None, ''),
 ('standard_type', 'varchar(250)', 'YES', 'MUL', None, ''),
 ('activity_comment', 'varchar(4000)', 'YES', '', None, ''),
 ('data_validity_comment', 'varchar(30)', 'YES', 'MUL', None, ''),
 ('potential_duplicate', 'tinyint(4)', 'YES', '', None, ''),
 ('pchembl_value', 'decimal(4,2)', 'YES', 'MUL', None, ''),
 ('bao_endpoint', 'varchar(11)', 'YES', 'MUL', None, ''),
 ('uo_units', 'varchar(10)', 'YES', '', None, ''),
 ('qudt_units', 'varchar(70)', 'YES', '', None, ''),
 ('toid', 'bigint(20)',

In [8]:
query= 'SELECT td.chembl_id, td.pref_name, act.standard_type, act.standard_relation, standard_value, standard_units, cs.canonical_smiles FROM compound_structures as cs \
INNER JOIN activities as act \
    ON cs.molregno = act.molregno \
INNER JOIN assays as asy \
    ON act.assay_id = asy.assay_id \
INNER JOIN target_dictionary as td \
    ON asy.tid = td.tid \
    where td.chembl_id = "CHEMBL612652" \
LIMIT 1000'

#CHEMBL2063 # protein
#CHEMBL612652 # parasite giardia
smiles = pd.read_sql(query, con=db_connection)
print(smiles)
data = pd.DataFrame(smiles)
data.to_csv('CHEMBL612652_giardia.csv', index = False)


        chembl_id             pref_name standard_type standard_relation  \
0    CHEMBL612652  Giardia intestinalis          ED50                 >   
1    CHEMBL612652  Giardia intestinalis          IC50                 =   
2    CHEMBL612652  Giardia intestinalis          IC50                 =   
3    CHEMBL612652  Giardia intestinalis          IC50                 =   
4    CHEMBL612652  Giardia intestinalis          IC50                 =   
..            ...                   ...           ...               ...   
817  CHEMBL612652  Giardia intestinalis          IC50                 =   
818  CHEMBL612652  Giardia intestinalis          IC50                 =   
819  CHEMBL612652  Giardia intestinalis          IC50                 =   
820  CHEMBL612652  Giardia intestinalis          IC50                 =   
821  CHEMBL612652  Giardia intestinalis          IC50                 =   

     standard_value standard_units  \
0              25.0        ug ml-1   
1             107.0    

In [10]:
db_connection.close()